In [1]:
import pandas as pd
import numpy as np
import sys
import time as tm
from allocation_solution_df import *
from vaccine_alloc_instance_df import *
from solvers.offline_maxutility_df import *
from solvers.online_maxutility_df import *
# from solvers.online_maxutility_df_1_by_del import *

In [2]:
#importing agents data
agents_type_df = pd.read_pickle('data/agents_type.pkl')
availability_df = pd.read_pickle('data/availability_df.pkl')
belongingness_df = pd.read_pickle('data/belongingness_df.pkl')

#importing hospitals data
daily_supply_df = pd.read_pickle('data/daily_supply.pkl')
overall_quota_df = pd.read_pickle('data/overall_quota.pkl')
Hospital_Data_Chennai_May_2022_df = pd.read_pickle('data/Hospital_Data_Chennai_May_2022.pkl')

In [3]:
n = 10000
c = 24
d = 30
q = sum(list(daily_supply_df['Total_available_capacity_per_day']))
availability = availability_df.values.tolist()
belongsToCatagory = belongingness_df.values.tolist()
Q_d = daily_supply_df['Total_available_capacity_per_day'].values.tolist()
Q_c = overall_quota_df['Total_available_capacity_per_hosp'].values.tolist()
# delta = agents_delta_value_df['Delta'].values.tolist()

In [4]:
delta_value = 0.95
alpha_values = [0.1, 0.5, 0.9]

In [5]:
agents_utility_value = []
for i in range(n):
    i_type = agents_type_df.iloc[i]['Age_group']
    i_alpha = alpha_values[i_type]
    i_utility = [i_alpha for x in range(d)]
    for j in range(d-1):
        i_utility[j+1] = i_utility[j]*delta_value
    agents_utility_value.append(i_utility)

In [6]:
categories = overall_quota_df['Center_ID'].values.tolist()
days = daily_supply_df['Date'].values.tolist()

# categories

In [7]:
Hospital_Data_Chennai_May_2022_df

,Date,Zip_Code,Center_ID,Center_Name,Total_available_capacity
0,01-05-2022,600034,603898,Kanchi Kamakoti Child Trust,9
1,01-05-2022,600089,617607,Srushti Hospital Pvt Ltd,86
3,01-05-2022,600116,855907,Srushti Clinic Porur,52
5,02-05-2022,600037,604136,MMRV Hospital,19
6,02-05-2022,600085,1037942,Srushti Hospital Kottupuram,29
...,...,...,...,...,...
162,30-05-2022,600089,617607,Srushti Hospital Pvt Ltd,58
164,30-05-2022,600093,580753,Sooriya Hospital,74
165,31-05-2022,600024,580742,Medway Hospitals,59
166,31-05-2022,600089,617607,Srushti Hospital Pvt Ltd,59


In [8]:
dict_Q_cxd = dict(zip(zip(Hospital_Data_Chennai_May_2022_df.Date, Hospital_Data_Chennai_May_2022_df.Center_ID),Hospital_Data_Chennai_May_2022_df.Total_available_capacity))
# dict_Q_cxd

In [9]:
dict_Q_cxd

{('01-05-2022 ', 603898): 9,
 ('01-05-2022 ', 617607): 86,
 ('01-05-2022 ', 855907): 52,
 ('02-05-2022 ', 604136): 19,
 ('02-05-2022 ', 1037942): 29,
 ('02-05-2022 ', 617607): 58,
 ('02-05-2022 ', 634485): 24,
 ('02-05-2022 ', 855907): 59,
 ('03-05-2022 ', 620736): 91,
 ('03-05-2022 ', 617607): 57,
 ('03-05-2022 ', 855907): 30,
 ('04-05-2022 ', 700076): 50,
 ('04-05-2022 ', 603579): 5,
 ('04-05-2022 ', 1037942): 30,
 ('04-05-2022 ', 617607): 60,
 ('04-05-2022 ', 603362): 9,
 ('04-05-2022 ', 855907): 60,
 ('05-05-2022 ', 700076): 50,
 ('05-05-2022 ', 855907): 30,
 ('06-05-2022 ', 700076): 50,
 ('06-05-2022 ', 620736): 299,
 ('06-05-2022 ', 611468): 30,
 ('06-05-2022 ', 1037942): 28,
 ('06-05-2022 ', 617607): 60,
 ('06-05-2022 ', 603362): 10,
 ('06-05-2022 ', 855907): 60,
 ('07-05-2022 ', 700076): 50,
 ('07-05-2022 ', 604384): 294,
 ('07-05-2022 ', 619367): 30,
 ('07-05-2022 ', 624694): 1,
 ('07-05-2022 ', 1037942): 30,
 ('07-05-2022 ', 617607): 60,
 ('07-05-2022 ', 855907): 30,
 ('08-05

In [10]:
Q_cxd = []
for cat in categories:
    c_d = []
    for day in days:
        if (day,cat) in dict_Q_cxd:
            c_d.append(dict_Q_cxd[(day,cat)])
        else:
            c_d.append(0)
    Q_cxd.append(c_d)

In [11]:
Q_cxd

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  59],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  74,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  30,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  9,
  0,
  10,
  0,
  0,
  0,
  0,
  10,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  50,
  0,
  0,
  50,
  0,
  0,
  0,
  39,
  0,
  0,
  50,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  40,
  0,
  0,
  0,
  0,
  0,

In [12]:
instance = VaccineAllocInstance(n,c,d,q,availability,belongsToCatagory,Q_d,Q_c,Q_cxd,agents_utility_value)

In [13]:
LP1 = LPOnlineMaxUtility(instance)
# LP1 = LPOnlineMaxUtility_1_by_del(instance)

In [14]:
start = tm.perf_counter()
sol = LP1.solve()
end = tm.perf_counter()
print("Total time taken: ",end - start)

Setting up LP...
Creating variables...
Creating constraints...
Solving LP...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/vishwa/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/f9181ca978da4c3e98231053a76ddf4c-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/f9181ca978da4c3e98231053a76ddf4c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5145 COLUMNS
At line 111870 RHS
At line 117011 BOUNDS
At line 156005 ENDATA
Problem MODEL has 5140 rows, 38993 columns and 72871 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-5137) rows, 1067 (-37926) columns and 1067 (-71804) elements
Perturbing problem by 0.001% of 0.9 - largest nonzero change 1.095314e-06 ( 0.0010172978%) - largest zero change 0
0  Obj -0 Dual inf 450.2981 (1067)
31  Obj 118.29982 Primal inf 295 (3)
62  Obj 118.29983 Primal inf 245 (3)


Setting up LP...
Creating variables...
Creating constraints...
Solving LP...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/vishwa/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/58f6bb0baeeb413081057737b294a8f3-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/58f6bb0baeeb413081057737b294a8f3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4399 COLUMNS
At line 97238 RHS
At line 101633 BOUNDS
At line 135501 ENDATA
Problem MODEL has 4394 rows, 33867 columns and 63365 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3114 (-1280) rows, 8306 (-25561) columns and 16608 (-46757) elements
Perturbing problem by 0.001% of 0.6615827 - largest nonzero change 5.3602691e-05 ( 0.068966081%) - largest zero change 0
0  Obj -0 Dual inf 1264.4731 (8306)
137  Obj 191.62175 Primal inf 74.99995 (50)
274  Obj 180.74275 P

Setting up LP...
Creating variables...
Creating constraints...
Solving LP...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/vishwa/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/502ef358e1d14c33af5692a8f5fc300d-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/502ef358e1d14c33af5692a8f5fc300d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 3396 COLUMNS
At line 75420 RHS
At line 78812 BOUNDS
At line 105073 ENDATA
Problem MODEL has 3391 rows, 26260 columns and 49154 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 909 (-2482) rows, 1844 (-24416) columns and 3658 (-45496) elements
Perturbing problem by 0.001% of 0.46200787 - largest nonzero change 5.1304806e-05 ( 0.098259795%) - largest zero change 0
0  Obj 3.0287183 Dual inf 122.72103 (1844)
6  Obj 23.766739
Optimal - objective value 23.767738
After 

Setting up LP...
Creating variables...
Creating constraints...
Solving LP...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/vishwa/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/43d2cfd131aa41febe7918ed0c09e52a-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/43d2cfd131aa41febe7918ed0c09e52a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2804 COLUMNS
At line 61750 RHS
At line 64550 BOUNDS
At line 86057 ENDATA
Problem MODEL has 2799 rows, 21506 columns and 40238 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 541 (-2258) rows, 1085 (-20421) columns and 2163 (-38075) elements
Perturbing problem by 0.001% of 0.17924296 - largest nonzero change 5.022228e-05 ( 0.14009554%) - largest zero change 0
0  Obj 2.0075212 Dual inf 39.571047 (1085)
19  Obj 14.549588
Optimal - objective value 14.554528
After Po

Setting up LP...
Creating variables...
Creating constraints...
Solving LP...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/vishwa/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/74e6822fa1d349d9955014ccf02490ed-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/74e6822fa1d349d9955014ccf02490ed-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1676 COLUMNS
At line 34231 RHS
At line 35903 BOUNDS
At line 47861 ENDATA
Problem MODEL has 1671 rows, 11957 columns and 22268 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-1671) rows, 0 (-11957) columns and 0 (-22268) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 0.025034409
After Postsolve, objective 0.025034409, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.02503440897 - 0 iterations time 0.002, Pres

In [15]:
solution = pd.DataFrame(sol.Allocation, columns=[str(i) for i in range(30)])

In [16]:
solution.to_pickle('Sols/Solution_online_df_alpha-0.1-0.5-0.9-delta-0.95.pkl')

In [17]:
print(sol.total_flow)
print(sol.total_utility)

7145.0
1550.2340219640914


In [18]:
LP2 = LPOfflineMaxUtility(instance)

In [15]:
start = tm.perf_counter()
sol2 = LP2.solve()
end = tm.perf_counter()
print("Total time taken: ",end - start)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/vishwa/.local/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/2dd6df03383e4153bc8990d6899f1ad3-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/2dd6df03383e4153bc8990d6899f1ad3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10780 COLUMNS
At line 3021508 RHS
At line 3032284 BOUNDS
At line 4042538 ENDATA
Problem MODEL has 10775 rows, 1010253 columns and 2011225 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 9570 (-1205) rows, 143826 (-866427) columns and 287486 (-1723739) elements
Perturbing problem by 0.001% of 0.9 - largest nonzero change 5.4857209e-05 ( 0.22375423%) - largest zero change 0
0  Obj -0 Dual inf 21668.725 (143826)
266  Obj 6825.0263 Primal inf 58488.195 (5184)
532  Obj 3613.3381 Primal inf 10914.998 (1822)
798  Obj 3358.384 Primal inf 9134.99

In [16]:
solution_off = pd.DataFrame(sol2.Allocation, columns=[str(i) for i in range(30)])

In [17]:
solution_off.to_pickle('Sols/Solution_offline_df_alpha-0.1-0.5-0.9-delta-0.95.pkl')

In [18]:
print(sol2.total_flow)
print(sol2.total_utility)


7192.0
1573.953169504987


In [ ]:
3567.949431626691/3580.6819391868944